In [1]:
from sqlalchemy.orm import joinedload
from sqlmodel import create_engine, Session, SQLModel, select
from dotenv import load_dotenv
import logging
import os
from modelos.post import Post, PostBaseWithUserCommentsTags

In [2]:
logging.basicConfig()
logging.getLogger("sqlalchemy.engine").setLevel(logging.INFO)

In [3]:
load_dotenv()
engine = create_engine(os.getenv("DATABASE_URL"))
SQLModel.metadata.create_all(engine)

INFO:sqlalchemy.engine.Engine:BEGIN (implicit)
INFO:sqlalchemy.engine.Engine:PRAGMA main.table_info("comment")
INFO:sqlalchemy.engine.Engine:[raw sql] ()
INFO:sqlalchemy.engine.Engine:PRAGMA main.table_info("user")
INFO:sqlalchemy.engine.Engine:[raw sql] ()
INFO:sqlalchemy.engine.Engine:PRAGMA main.table_info("tag")
INFO:sqlalchemy.engine.Engine:[raw sql] ()
INFO:sqlalchemy.engine.Engine:PRAGMA main.table_info("posttag")
INFO:sqlalchemy.engine.Engine:[raw sql] ()
INFO:sqlalchemy.engine.Engine:PRAGMA main.table_info("post")
INFO:sqlalchemy.engine.Engine:[raw sql] ()
INFO:sqlalchemy.engine.Engine:COMMIT


In [4]:
with Session(engine) as session:
    #post = session.get(Post, 1)
    statement = select(Post).where(Post.id == 1).options(joinedload(Post.user), joinedload(Post.comments), joinedload(Post.tags))
    #statement = select(Post).where(Post.id == 1)
    post = session.exec(statement).first()
    print('---------------------------------------------------------')
    print('post:', post)
    print('post.user:', post.user)
    print('post.comments:', post.comments)
    print('post.tags:', post.tags)
    print('---------------------------------------------------------')
    print(post.model_dump_json(indent=2))
    print('---------------------------------------------------------')
    post2 = PostBaseWithUserCommentsTags(
        **{k: v for k, v in post.__dict__.items() if k in PostBaseWithUserCommentsTags.model_fields.keys()}
    )
    print(post2.model_dump_json(indent=2))

INFO:sqlalchemy.engine.Engine:BEGIN (implicit)
INFO:sqlalchemy.engine.Engine:SELECT post.id, post.title, post.content, post.created_at, post.updated_at, post.user_id, user_1.id AS id_1, user_1.name, user_1.email, comment_1.id AS id_2, comment_1.content AS content_1, comment_1.created_at AS created_at_1, comment_1.updated_at AS updated_at_1, comment_1.post_id, comment_1.user_id AS user_id_1, tag_1.id AS id_3, tag_1.name AS name_1 
FROM post LEFT OUTER JOIN user AS user_1 ON user_1.id = post.user_id LEFT OUTER JOIN comment AS comment_1 ON post.id = comment_1.post_id LEFT OUTER JOIN (posttag AS posttag_1 JOIN tag AS tag_1 ON tag_1.id = posttag_1.tag_id) ON post.id = posttag_1.post_id 
WHERE post.id = ?
INFO:sqlalchemy.engine.Engine:[generated in 0.00124s] (1,)
INFO:sqlalchemy.engine.Engine:ROLLBACK


---------------------------------------------------------
post: title='Post 1' created_at=datetime.datetime(2025, 1, 12, 15, 7, 13, 436734) user_id=1 id=1 content='Conteúdo do 1' updated_at=datetime.datetime(2025, 1, 12, 15, 7, 13, 436745)
post.user: email='joão@gmail.com' name='João' id=1
post.comments: [Comment(post_id=1, user_id=1, created_at=datetime.datetime(2025, 1, 12, 19, 24, 31, 403369), id=1, content='Commentário 1', updated_at=datetime.datetime(2025, 1, 12, 19, 24, 31, 403382)), Comment(post_id=1, user_id=2, created_at=datetime.datetime(2025, 1, 12, 19, 25, 4, 426501), id=2, content='Commentário 2', updated_at=datetime.datetime(2025, 1, 12, 19, 25, 4, 426516)), Comment(post_id=1, user_id=3, created_at=datetime.datetime(2025, 1, 12, 19, 25, 10, 911482), id=3, content='Commentário 3', updated_at=datetime.datetime(2025, 1, 12, 19, 25, 10, 911493)), Comment(post_id=1, user_id=4, created_at=datetime.datetime(2025, 1, 12, 19, 25, 28, 470366), id=4, content='Commentário 4', updated